In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import matplotlib.dates as mdates
import numpy as np
import numpy as np

from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd

In [ ]:
# '''
# query ($fromdate: Int!){
#   poolHourDatas(
#     where: {pool: "''' + str(Adress) + '''",, periodStartUnix_gt: $fromdate},
#     orderBy: periodStartUnix,
#     orderDirection: desc,
#     first: 1000
#   ) {
#     periodStartUnix
#     liquidity
#     low
#     pool {
#       totalValueLockedUSD
#       totalValueLockedToken1
#       totalValueLockedToken0
#       token0 {
#         decimals
#       }
#       token1 {
#         decimals
#       }
#     }
#     close
#     feeGrowthGlobal0X128
#     feeGrowthGlobal1X128
#   }
# }
# '''

In [19]:
def graph(Adress, fromdate):

    sample_transport = RequestsHTTPTransport(
        url='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3',
        verify=True,
        retries=5,
    )
    client = Client(
        transport=sample_transport
    )

    print(fromdate)
    
    query = gql(
        '''
        query ($fromdate: Int!){
            poolHourDatas(
                where: {pool: "''' + str(Adress) + '''", periodStartUnix_gt: $fromdate},
                orderBy: periodStartUnix,
                orderDirection: desc,
                first: 1000
            ) {
                periodStartUnix
                liquidity
                low
                pool {
                    totalValueLockedUSD
                    totalValueLockedToken1
                    totalValueLockedToken0
                    token0 {
                        decimals
                    }
                    token1 {
                        decimals
                    }
                }
                close
                feeGrowthGlobal0X128
                feeGrowthGlobal1X128
            }
        }
        '''
    )

    params = {
        "fromdate": fromdate
    }

    response = client.execute(query, variable_values=params)
    dpd = pd.json_normalize(response['poolHourDatas'])
    dpd = dpd.astype(float)
    return dpd

In [14]:
def get_amount0(sqrtA, sqrtB, liquidity, decimals):
    if sqrtA > sqrtB:
        (sqrtA, sqrtB) = (sqrtB, sqrtA)

    amount0 = ((liquidity * 2 ** 96 * (sqrtB - sqrtA) / sqrtB / sqrtA) / 10 ** decimals)

    return amount0


def get_amount1(sqrtA, sqrtB, liquidity, decimals):
    if sqrtA > sqrtB:
        (sqrtA, sqrtB) = (sqrtB, sqrtA)

    amount1 = liquidity * (sqrtB - sqrtA) / 2 ** 96 / 10 ** decimals

    return amount1


def get_amounts(asqrt, asqrtA, asqrtB, liquidity, decimal0, decimal1):
    sqrt = (np.sqrt(asqrt * 10 ** (decimal1 - decimal0))) * (2 ** 96)
    sqrtA = np.sqrt(asqrtA * 10 ** (decimal1 - decimal0)) * (2 ** 96)
    sqrtB = np.sqrt(asqrtB * 10 ** (decimal1 - decimal0)) * (2 ** 96)

    if sqrtA > sqrtB:
        (sqrtA, sqrtB) = (sqrtB, sqrtA)

    if sqrt <= sqrtA:

        amount0 = get_amount0(sqrtA, sqrtB, liquidity, decimal0)
        return amount0, 0

    elif sqrtB > sqrt > sqrtA:
        amount0 = get_amount0(sqrt, sqrtB, liquidity, decimal0)

        amount1 = get_amount1(sqrtA, sqrt, liquidity, decimal1)

        return amount0, amount1

    else:
        amount1 = get_amount1(sqrtA, sqrtB, liquidity, decimal1)
        return 0, amount1


'''get_liquidity function'''


# Use 'get_liquidity' function to calculate liquidity as a function of amounts and price range
def get_liquidity0(sqrtA, sqrtB, amount0, decimals):
    if sqrtA > sqrtB:
        (sqrtA, sqrtB) = (sqrtB, sqrtA)

    liquidity = amount0 / ((2 ** 96 * (sqrtB - sqrtA) / sqrtB / sqrtA) / 10 ** decimals)
    return liquidity


def get_liquidity1(sqrtA, sqrtB, amount1, decimals):
    if sqrtA > sqrtB:
        (sqrtA, sqrtB) = (sqrtB, sqrtA)

    liquidity = amount1 / ((sqrtB - sqrtA) / 2 ** 96 / 10 ** decimals)
    return liquidity


def get_liquidity(asqrt, asqrtA, asqrtB, amount0, amount1, decimal0, decimal1):
    sqrt = (np.sqrt(asqrt * 10 ** (decimal1 - decimal0))) * (2 ** 96)
    sqrtA = np.sqrt(asqrtA * 10 ** (decimal1 - decimal0)) * (2 ** 96)
    sqrtB = np.sqrt(asqrtB * 10 ** (decimal1 - decimal0)) * (2 ** 96)

    if sqrtA > sqrtB:
        (sqrtA, sqrtB) = (sqrtB, sqrtA)

    if sqrt <= sqrtA:

        liquidity0 = get_liquidity0(sqrtA, sqrtB, amount0, decimal0)
        return liquidity0

    elif sqrtB > sqrt > sqrtA:

        liquidity0 = get_liquidity0(sqrt, sqrtB, amount0, decimal0)
        liquidity1 = get_liquidity1(sqrtA, sqrt, amount1, decimal1)
        liquidity = liquidity0 if liquidity0 < liquidity1 else liquidity1
        return liquidity

    else:
        liquidity1 = get_liquidity1(sqrtA, sqrtB, amount1, decimal1)
        return liquidity1


In [15]:
import pandas as pd


def chart1(dpd, base, myliquidity):
    if base == 0:
        dpd['feeV'] = (dpd['myfee0']) + (dpd['myfee1'] * dpd['close'])
        dpd['amountV'] = (dpd['amount0']) + (dpd['amount1'] * dpd['close'])
        dpd['amountunb'] = (dpd['amount0unb']) + (dpd['amount1unb'] * dpd['close'])
        dpd['fgV'] = (dpd['fee0token']) + (dpd['fee1token'] * dpd['close'])
        dpd['feeusd'] = dpd['feeV'] * (dpd['pool.totalValueLockedUSD'].iloc[0] / (
                    dpd['pool.totalValueLockedToken1'].iloc[0] * dpd['close'].iloc[0] + (
            dpd['pool.totalValueLockedToken0'].iloc[0])))


    else:

        dpd['feeV'] = (dpd['myfee0'] / dpd['close']) + dpd['myfee1']
        dpd['amountV'] = (dpd['amount0'] / dpd['close']) + dpd['amount1']
        dpd['feeVbase0'] = dpd['myfee0'] + (dpd['myfee1'] * dpd['close'])
        dpd['amountunb'] = (dpd['amount0unb'] / dpd['close']) + dpd['amount1unb']
        dpd['fgV'] = (dpd['fee0token'] / dpd['close']) + dpd['fee1token']
        dpd['feeusd'] = dpd['feeV'] * (dpd['pool.totalValueLockedUSD'].iloc[0] / (
                    dpd['pool.totalValueLockedToken1'].iloc[0] + (
                        dpd['pool.totalValueLockedToken0'].iloc[0] / dpd['close'].iloc[0])))

    dpd['date'] = pd.to_datetime(dpd['periodStartUnix'], unit='s')

    # 1 Chart

    # dpd['fgV']= (dpd['fg0'] / dpd['close'].iloc[0] + dpd['fg1'])
    # rint(dpd['fg1']/dpd['amount1unb'])

    data = dpd[
        ['date', 'myfee0', 'myfee1', 'fgV', 'feeV', 'feeusd', 'amountV', 'ActiveLiq', 'amountunb', 'amount0', 'amount1',
         'close']]
    data = data.fillna(0)

    temp = data.resample('D', on='date').sum()
    final1 = temp[['myfee0', 'myfee1', 'feeV', 'fgV', 'feeusd']].copy()

    temp2 = data.resample('D', on='date').mean()
    final1['ActiveLiq'] = temp2['ActiveLiq'].copy()

    temp3 = data.resample('D', on='date').first()
    final1[['amountV', 'amountunb']] = temp3[['amountV', 'amountunb']].copy()
    temp4 = data.resample('D', on='date').last()
    final1[['amountVlast']] = temp4[['amountV']]

    final1['S1%'] = final1['feeV'] / final1['amountV'] * 100  # *365
    final1['unb%'] = final1['fgV'] / final1['amountunb'] * 100  # *365
    final1['multiplier'] = final1['S1%'] / final1['unb%']
    final1['feeunb'] = final1['amountV'] * final1['unb%'] / 100
    final1.to_csv("chart1.csv", sep=";")

    print(final1[['feeunb', 'feeV', 'feeusd', 'amountV', 'ActiveLiq', 'S1%', 'unb%', 'ActiveLiq']])

    print('------------------------------------------------------------------')
    print("this position returned", final1['feeV'].sum() / final1['amountV'].iloc[0] * 100, "in ", len(final1.index),
          " days, for an apr of ", final1['feeV'].sum() / final1['amountV'].iloc[0] * 365 / len(final1.index) * 100)
    print("a base  position returned", final1['feeunb'].sum() / final1['amountV'].iloc[0] * 100, "in ",
          len(final1.index), " days, for an apr of ",
          final1['feeunb'].sum() / final1['amountV'].iloc[0] * 365 / len(final1.index) * 100)

    print("fee in token 1 and token 2", dpd['myfee0'].sum(), dpd['myfee1'].sum())
    print("totalFee in USD", final1['feeusd'].sum())
    print('Your liquidity was active for:', final1['ActiveLiq'].mean())
    forecast = (dpd['feeVbase0'].sum() * myliquidity * final1['ActiveLiq'].mean())
    print(dpd['feeVbase0'])
    print('forecast: ', forecast)
    print('------------------------------------------------------------------')
    # 1 chart e' completo

    # 2 chart

    final2 = temp3[['amountV', 'amount0', 'amount1', 'close']].copy()
    final2['feeV'] = temp['feeV'].copy()
    final2[['amountVlast']] = temp4[['amountV']]

    final2['HODL'] = final2['amount0'].iloc[0] / final2['close'] + final2['amount1'].iloc[0]

    final2['IL'] = final2['amountVlast'] - final2['HODL']
    final2['ActiveLiq'] = temp2['ActiveLiq'].copy()
    final2['feecumsum'] = final2['feeV'].cumsum()
    final2['PNL'] = final2['feecumsum'] + final2['IL']  # -Bfinal['gas']

    final2['HODLnorm'] = final2['HODL'] / final2['amountV'].iloc[0] * 100
    final2['ILnorm'] = final2['IL'] / final2['amountV'].iloc[0] * 100
    final2['PNLnorm'] = final2['PNL'] / final2['amountV'].iloc[0] * 100
    final2['feecumsumnorm'] = final2['feecumsum'] / final2['amountV'].iloc[0] * 100
    ch2 = final2[['amountV', 'feecumsum']]
    ch3 = final2[['ILnorm', 'PNLnorm', 'feecumsumnorm']]

    final2.to_csv("chart2.csv", sep=";")
    print(ch2)
    print(ch3)

    # final3=data
    final3 = pd.DataFrame()
    final3['amountV'] = data['amountV']

    final3['amountVlast'] = data['amountV'].shift(-1)
    final3['date'] = data['date']
    final3['HODL'] = data['amount0'].iloc[0] / data['close'] + data['amount1'].iloc[0]

    final3['amountVlast'].iloc[-1] = final3['HODL'].iloc[-1]
    final3['IL'] = final3['amountVlast'] - final3['HODL']
    final3['feecumsum'] = data['feeV'][::-1].cumsum()
    final3['PNL'] = final3['feecumsum'] + final3['IL']
    final3['HODLnorm'] = final3['HODL'] / final3['amountV'].iloc[0] * 100
    final3['ILnorm'] = final3['IL'] / final3['amountV'].iloc[0] * 100
    final3['PNLnorm'] = final3['PNL'] / final3['amountV'].iloc[0] * 100
    final3['feecumsumnorm'] = final3['feecumsum'] / final3['amountV'].iloc[0] * 100

    ch2 = final3[['amountV', 'feecumsum']]
    ch3 = final3[['ILnorm', 'PNLnorm', 'feecumsumnorm']]

    print(ch2)
    print(ch3)


In [20]:
# network 1 ETH, 2 ARB, 3 OPT

# Adress = "0x93f267fd92b432bebf4da4e13b8615bb8eb2095c"  # snx eth
# Adress = "0xcb0c5d9d92f4f2f80cce7aa271a1e148c226e19d"  # Rai Dai
Adress = "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8"  # ETH USDC Ethereum
# Adress = "0x2e9c575206288f2219409289035facac0b670c2f"  # ETH DAI Optimism
# Adress= "0x68f180fcce6836688e9084f035309e29bf0a2095" #  WBTC DAI
startfrom = 1632081600

In [21]:
dpd = graph(Adress, startfrom)
dpd.head()

1632081600


,periodStartUnix,liquidity,low,close,feeGrowthGlobal0X128,feeGrowthGlobal1X128,pool.totalValueLockedUSD,pool.totalValueLockedToken1,pool.totalValueLockedToken0,pool.token0.decimals,pool.token1.decimals
0,1.704434e+09,5.612540e+18,2244.845558,2257.353021,3.267627e+33,1.536735e+42,2.910011e+08,65435.237115,1.432906e+08,6.0,18.0
1,1.704431e+09,5.678531e+18,2243.790255,2243.790255,3.267473e+33,1.536691e+42,2.910011e+08,65435.237115,1.432906e+08,6.0,18.0
2,1.704427e+09,5.678531e+18,2243.757056,2243.767036,3.267473e+33,1.536691e+42,2.910011e+08,65435.237115,1.432906e+08,6.0,18.0
3,1.704424e+09,5.678531e+18,2242.247926,2244.945740,3.267472e+33,1.536686e+42,2.910011e+08,65435.237115,1.432906e+08,6.0,18.0
4,1.704420e+09,5.678440e+18,2240.831770,2242.263377,3.267428e+33,1.536686e+42,2.910011e+08,65435.237115,1.432906e+08,6.0,18.0


In [8]:
decimal0 = dpd.iloc[0]['pool.token0.decimals']
decimal1 = dpd.iloc[0]['pool.token1.decimals']
decimal = decimal1 - decimal0

In [9]:
dpd['fg0'] = ((dpd['feeGrowthGlobal0X128']) / (2 ** 128)) / (10 ** decimal0)
dpd['fg1'] = ((dpd['feeGrowthGlobal1X128']) / (2 ** 128)) / (10 ** decimal1)

In [10]:
mini = 3273.4
maxi = 4038.3
target = 2711.53  # 1 / dpd['close'].iloc[-1]
base = 1

# mini = 1 / 3215    #optimism
# maxi = 1 / 2903.3 
# target = 1 #in base 0
# base = 0

# mini =  2892.9          #ethereum
# maxi =  3235  
# target = 5910 #in base 0
# base = 0

# mini =  1/4048         #ethereum
# maxi =  1/ 2892 
# target = 5910 / dpd['close'].iloc[0] #in base 0
# base = 1

In [11]:
# Calculate F0G and F1G (fee earned by an unbounded unit of liquidity in one period)

dpd['fg0shift'] = dpd['fg0'].shift(-1)
dpd['fg1shift'] = dpd['fg1'].shift(-1)
dpd['fee0token'] = dpd['fg0'] - dpd['fg0shift']
dpd['fee1token'] = dpd['fg1'] - dpd['fg1shift']

In [12]:
# calculate my liquidity

SMIN = np.sqrt(mini * 10 ** (decimal))
SMAX = np.sqrt(maxi * 10 ** (decimal))

if base == 0:

    sqrt0 = np.sqrt(dpd['close'].iloc[-1] * 10 ** (decimal))
    dpd['price0'] = dpd['close']

else:

    sqrt0 = np.sqrt(1 / dpd['close'].iloc[-1] * 10 ** (decimal))
    dpd['price0'] = 1 / dpd['close']

if SMIN < sqrt0 < SMAX:

    deltaL = target / ((sqrt0 - SMIN) + (((1 / sqrt0) - (1 / SMAX)) * (dpd['price0'].iloc[-1] * 10 ** (decimal))))
    amount1 = deltaL * (sqrt0 - SMIN)
    amount0 = deltaL * ((1 / sqrt0) - (1 / SMAX)) * 10 ** (decimal)

elif sqrt0 < SMIN:

    deltaL = target / (((1 / SMIN) - (1 / SMAX)) * (dpd['price0'].iloc[-1]))
    amount1 = 0
    amount0 = deltaL * ((1 / SMIN) - (1 / SMAX))

else:
    deltaL = target / (SMAX - SMIN)
    amount1 = deltaL * (SMAX - SMIN)
    amount0 = 0
    
print("Amounts:", amount0, amount1)

Amounts: 5593241.785659055 0


In [16]:
# print(dpd['price0'].iloc[-1],mini,maxi)
# print((dpd['price0'].iloc[-1],mini,maxi,amount0,amount1,decimal0,decimal1))
myliquidity = get_liquidity(dpd['price0'].iloc[-1], mini, maxi, amount0, amount1, decimal0, decimal1)

print("OK myliquidity", myliquidity)

OK myliquidity 3.210593910812648e+21


In [17]:
# Calculate ActiveLiq

dpd['ActiveLiq'] = 0
dpd['amount0'] = 0
dpd['amount1'] = 0
dpd['amount0unb'] = 0
dpd['amount1unb'] = 0

if base == 0:

    for i, row in dpd.iterrows():
        if dpd['high'].iloc[i] > mini and dpd['low'].iloc[i] < maxi:
            dpd.iloc[i, dpd.columns.get_loc('ActiveLiq')] = (min(maxi, dpd['high'].iloc[i]) - max(dpd['low'].iloc[i],
                                                                                                  mini)) / (
                                                                        dpd['high'].iloc[i] - dpd['low'].iloc[i]) * 100
        else:
            dpd.iloc[i, dpd.columns.get_loc('ActiveLiq')] = 0

        amounts = get_amounts(dpd['price0'].iloc[i], mini, maxi, myliquidity, decimal0, decimal1)
        dpd.iloc[i, dpd.columns.get_loc('amount0')] = amounts[1]
        dpd.iloc[i, dpd.columns.get_loc('amount1')] = amounts[0]

        amountsunb = get_amounts((dpd['price0'].iloc[i]), 1.0001 ** (-887220), 1.0001 ** 887220, 1, decimal0, decimal1)
        dpd.iloc[i, dpd.columns.get_loc('amount0unb')] = amountsunb[1]
        dpd.iloc[i, dpd.columns.get_loc('amount1unb')] = amountsunb[0]

else:

    for i, row in dpd.iterrows():

        if (1 / dpd['low'].iloc[i]) > mini and (1 / dpd['high'].iloc[i]) < maxi:
            dpd.iloc[i, dpd.columns.get_loc('ActiveLiq')] = (min(maxi, 1 / dpd['low'].iloc[i]) - max(
                1 / dpd['high'].iloc[i], mini)) / ((1 / dpd['low'].iloc[i]) - (1 / dpd['high'].iloc[i])) * 100
        else:
            dpd.iloc[i, dpd.columns.get_loc('ActiveLiq')] = 0

        amounts = get_amounts((dpd['price0'].iloc[i] * 10 ** (decimal)), mini, maxi, myliquidity, decimal0,
                                        decimal1)
        dpd.iloc[i, dpd.columns.get_loc('amount0')] = amounts[0]
        dpd.iloc[i, dpd.columns.get_loc('amount1')] = amounts[1]

        amountsunb = get_amounts((dpd['price0'].iloc[i]), 1.0001 ** (-887220), 1.0001 ** 887220, 1, decimal0,
                                           decimal1)
        dpd.iloc[i, dpd.columns.get_loc('amount0unb')] = amountsunb[0]
        dpd.iloc[i, dpd.columns.get_loc('amount1unb')] = amountsunb[1]

In [18]:
## Final fee calculation

dpd['myfee0'] = dpd['fee0token'] * myliquidity * dpd['ActiveLiq'] / 100
dpd['myfee1'] = dpd['fee1token'] * myliquidity * dpd['ActiveLiq'] / 100

# print(dpd)

a = chart1(dpd, base, myliquidity)

                  feeunb  feeV  feeusd       amountV  ActiveLiq  S1%  \
date                                                                   
2023-11-23  8.184650e+05   0.0     0.0  2.033585e+10        0.0  0.0   
2023-11-24  1.375497e+06   0.0     0.0  2.033585e+10        0.0  0.0   
2023-11-25  2.189479e+05   0.0     0.0  2.033585e+10        0.0  0.0   
2023-11-26  8.250091e+05   0.0     0.0  2.033585e+10        0.0  0.0   
2023-11-27  1.283335e+06   0.0     0.0  2.033585e+10        0.0  0.0   
2023-11-28  1.108599e+06   0.0     0.0  2.033585e+10        0.0  0.0   
2023-11-29  7.409130e+05   0.0     0.0  2.033585e+10        0.0  0.0   
2023-11-30  5.174683e+05   0.0     0.0  2.033585e+10        0.0  0.0   
2023-12-01  9.669251e+05   0.0     0.0  2.033585e+10        0.0  0.0   
2023-12-02  1.253200e+06   0.0     0.0  2.033585e+10        0.0  0.0   
2023-12-03  7.623924e+05   0.0     0.0  2.033585e+10        0.0  0.0   
2023-12-04  2.674375e+06   0.0     0.0  2.033585e+10        0.0 

/var/folders/s0/gcg7k5ws6hxfgnzww6_ryn0c0000gn/T/ipykernel_7191/886535243.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final3['amountVlast'].iloc[-1] = final3['HODL'].iloc[-1]
